In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.gridspec as gridspec
from numpy import random
import scipy.stats as scipy
import time
import torch.nn as nn
import torch.nn.functional as F

from IPython.display import display

from sklearn.metrics import accuracy_score

In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from enum import Enum 
from torchmetrics import Accuracy
import torch
%matplotlib inline

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._reg

In [3]:
df = pd.read_csv('best_dataset.csv')
df.tail()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTR,HST,AST,HC,AC,...,league_id,season,stage,date,home_team_api_id,away_team_api_id,home_team_long_name,home_team_short_name,away_team_long_name,away_team_short_name
536,6067,E0,2016-05-11 00:00:00,Liverpool,Chelsea,D,9,7,7,3,...,1729,2015/2016,30,2016-05-11 00:00:00,8650,8455,Liverpool,LIV,Chelsea,CHE
537,6069,E0,2016-05-11 00:00:00,Sunderland,Everton,H,8,6,5,5,...,1729,2015/2016,30,2016-05-11 00:00:00,8472,8668,Sunderland,SUN,Everton,EVE
538,6070,E0,2016-05-15 00:00:00,Arsenal,Aston Villa,H,7,2,5,4,...,1729,2015/2016,38,2016-05-15 00:00:00,9825,10252,Arsenal,ARS,Aston Villa,AVL
539,6074,E0,2016-05-15 00:00:00,Southampton,Crystal Palace,H,5,4,2,5,...,1729,2015/2016,38,2016-05-15 00:00:00,8466,9826,Southampton,SOU,Crystal Palace,CRY
540,6077,E0,2016-05-15 00:00:00,Watford,Sunderland,D,6,4,7,4,...,1729,2015/2016,38,2016-05-15 00:00:00,9817,8472,Watford,WAT,Sunderland,SUN


In [4]:
df = df.dropna(inplace=False)

In [5]:
df = df.drop(['Unnamed: 0', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'match_api_id', 'country_id', 'league_id', 'stage', 'season', 'date', \
              'home_team_api_id', 'home_team_long_name', 'home_team_short_name', \
              'away_team_api_id', 'away_team_long_name', 'away_team_short_name', \
              'League_1729.0', 'League_4769.0', 'League_7809.0', 'League_10257.0', 'League_13274.0', 'League_19694.0', 'League_21518.0'], axis=1)
df.tail()

,HST,AST,HC,AC,HAS,HDS,AAS,ADS,Result,home_team_buildUpPlaySpeed,...,away_player_3_overall_rating,away_player_4_overall_rating,away_player_5_overall_rating,away_player_6_overall_rating,away_player_7_overall_rating,away_player_8_overall_rating,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,outcome
536,9,7,7,3,1.233395,0.690260,1.389295,0.619937,0,66.0,...,82.0,77.0,77.0,86.0,83.0,83.0,88.0,83.0,74.0,Draw
537,8,6,5,5,0.587536,0.883299,0.956420,0.896425,1,43.0,...,77.0,75.0,82.0,80.0,79.0,80.0,80.0,77.0,82.0,Win
538,7,2,5,4,1.429960,0.739982,0.997368,1.082190,1,59.0,...,74.0,61.0,73.0,76.0,74.0,75.0,72.0,76.0,75.0,Win
539,5,4,2,5,0.568096,0.552793,0.236911,0.259207,1,52.0,...,71.0,74.0,75.0,76.0,76.0,75.0,71.0,71.0,78.0,Win
540,6,4,7,4,0.084242,0.128693,0.640538,0.915866,0,61.0,...,74.0,75.0,59.0,75.0,60.0,79.0,67.0,74.0,76.0,Draw


In [6]:
df = df.drop('Result', axis=1)
df.tail()

,HST,AST,HC,AC,HAS,HDS,AAS,ADS,home_team_buildUpPlaySpeed,home_team_buildUpPlayPassing,...,away_player_3_overall_rating,away_player_4_overall_rating,away_player_5_overall_rating,away_player_6_overall_rating,away_player_7_overall_rating,away_player_8_overall_rating,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating,outcome
536,9,7,7,3,1.233395,0.690260,1.389295,0.619937,66.0,45.0,...,82.0,77.0,77.0,86.0,83.0,83.0,88.0,83.0,74.0,Draw
537,8,6,5,5,0.587536,0.883299,0.956420,0.896425,43.0,51.0,...,77.0,75.0,82.0,80.0,79.0,80.0,80.0,77.0,82.0,Win
538,7,2,5,4,1.429960,0.739982,0.997368,1.082190,59.0,30.0,...,74.0,61.0,73.0,76.0,74.0,75.0,72.0,76.0,75.0,Win
539,5,4,2,5,0.568096,0.552793,0.236911,0.259207,52.0,46.0,...,71.0,74.0,75.0,76.0,76.0,75.0,71.0,71.0,78.0,Win
540,6,4,7,4,0.084242,0.128693,0.640538,0.915866,61.0,52.0,...,74.0,75.0,59.0,75.0,60.0,79.0,67.0,74.0,76.0,Draw


In [9]:
##########################
### SETTINGS
##########################

RANDOM_SEED = 1
BATCH_SIZE = 16
drop_proba = 0.45
weight_decay = 0.001
num_hidden1 = 128
num_hidden2 = 64
num_hidden3 = 32
NUM_EPOCHS = 100
NUM_FEATURES = 61
learning_rate = 0.0008
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
outcomes = df['outcome'].values

def encode_labels(y):
    y[y == 'Win'] = 2
    y[y == 'Defeat'] = 0
    y[y == 'Draw'] = 1
    return y

outcomes = encode_labels(outcomes)

outcomes = pd.DataFrame(outcomes)
outcomes

,0
0,0
1,2
2,2
3,2
4,0
...,...
536,1
537,2
538,2
539,2


In [11]:
feature = df.drop('outcome', axis=1)
feature = pd.DataFrame(feature)
feature.tail()

,HST,AST,HC,AC,HAS,HDS,AAS,ADS,home_team_buildUpPlaySpeed,home_team_buildUpPlayPassing,...,away_player_2_overall_rating,away_player_3_overall_rating,away_player_4_overall_rating,away_player_5_overall_rating,away_player_6_overall_rating,away_player_7_overall_rating,away_player_8_overall_rating,away_player_9_overall_rating,away_player_10_overall_rating,away_player_11_overall_rating
536,9,7,7,3,1.233395,0.690260,1.389295,0.619937,66.0,45.0,...,82.0,82.0,77.0,77.0,86.0,83.0,83.0,88.0,83.0,74.0
537,8,6,5,5,0.587536,0.883299,0.956420,0.896425,43.0,51.0,...,63.0,77.0,75.0,82.0,80.0,79.0,80.0,80.0,77.0,82.0
538,7,2,5,4,1.429960,0.739982,0.997368,1.082190,59.0,30.0,...,58.0,74.0,61.0,73.0,76.0,74.0,75.0,72.0,76.0,75.0
539,5,4,2,5,0.568096,0.552793,0.236911,0.259207,52.0,46.0,...,76.0,71.0,74.0,75.0,76.0,76.0,75.0,71.0,71.0,78.0
540,6,4,7,4,0.084242,0.128693,0.640538,0.915866,61.0,52.0,...,71.0,74.0,75.0,59.0,75.0,60.0,79.0,67.0,74.0,76.0


In [13]:
train_data, test_data, train_targets, test_targets = train_test_split(feature, outcomes, test_size=0.2, random_state=123)

In [14]:
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert data and targets to tensors with desired data type
        data_tensor = torch.tensor(self.data.iloc[idx, :].values, dtype=torch.float)
        target_tensor = torch.tensor(self.targets.iloc[idx], dtype=torch.long)  # Assuming targets are integer labels
        
        # Return data and targets as tensors
        return data_tensor, target_tensor


In [15]:
print(f"X Train Set: {train_data.shape}")
print(f"X Test Set: {test_data.shape}")
print(f"y Train Set: {train_targets.shape}")
print(f"y Test Set: {test_targets.shape}")

X Train Set: (432, 61)
X Test Set: (109, 61)
y Train Set: (432, 1)
y Test Set: (109, 1)


In [16]:
train_dataset = CustomDataset(train_data, train_targets)
test_dataset = CustomDataset(test_data, test_targets)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

## Model

In [17]:
class MLP(torch.nn.Module):

    def __init__(self, num_features, num_hidden1, num_hidden2, num_hidden3, num_classes, drop_proba):
        super().__init__()

        self.num_classes = num_classes
        self.num_features = num_features
        self.num_hidden1 = num_hidden1
        self.num_hidden2 = num_hidden2
        self.num_hidden3 = num_hidden3

        self.my_network = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Flatten(),
            torch.nn.Linear(num_features, num_hidden1),
            torch.nn.BatchNorm1d(num_hidden1),
            torch.nn.ReLU(),
            torch.nn.Dropout(drop_proba),

            # 2nd hidden layer
            torch.nn.Linear(num_hidden1, num_hidden2),
            torch.nn.BatchNorm1d(num_hidden2),
            torch.nn.ReLU(),
            torch.nn.Dropout(drop_proba),

            # 3rd hidden layer
            torch.nn.Linear(num_hidden2, num_hidden3),
            torch.nn.BatchNorm1d(num_hidden3),
            torch.nn.ReLU(),
            torch.nn.Dropout(drop_proba),

            # 5th hidden layer
            # torch.nn.Linear(num_hidden4, num_hidden5),
            # torch.nn.BatchNorm1d(num_hidden5),
            # torch.nn.ReLU(),
            # torch.nn.Dropout(drop_proba),


            # output hidden layer
            torch.nn.Linear(num_hidden3, num_classes),
        )
        
        # for m in self.modules():
        #     if isinstance(m, torch.nn.Linear):
        #         torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        #         if m.bias is not None:
        #             m.bias.detach().zero_()


    def forward(self, x):
        logits = self.my_network(x)
        probas = torch.softmax(logits, dim=1)
        return logits, probas

In [18]:
def compute_accuracy(model, X_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in X_loader:
            inputs = inputs.view(-1, NUM_FEATURES).to(DEVICE)
            targets = targets.view(-1,1).to(DEVICE)
            logits, probas = model.forward(inputs)
            predicted = torch.argmax(logits, dim=1)
            total += targets.size(0)
            correct += (predicted == targets.view(-1)).sum().item()
        accuracy = correct / total * 100
        return accuracy


def compute_loss(model, X_loader):
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        for inputs, targets in X_loader:
            inputs = inputs.view(-1, NUM_FEATURES).to(DEVICE)
            targets = targets.view(-1,1).to(DEVICE)
            logits, probas = model.forward(inputs)
            loss = F.cross_entropy(logits, targets.view(-1))
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(X_loader)
        return avg_loss

In [21]:
## Model initialization

torch.manual_seed(RANDOM_SEED)
model = MLP(num_features=NUM_FEATURES,
            num_hidden1=num_hidden1,
            num_hidden2=num_hidden2,
            num_hidden3=num_hidden3,
            num_classes=3,
            drop_proba=drop_proba)

model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       mode='max',
                                                       verbose=True)


train_acc_data = []
train_cost_data = []
test_acc_data = []
test_cost_data = []

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [22]:
model.train()
for epoch in range(NUM_EPOCHS):
    model.train()

    for inputs, targets in train_loader:
    
        inputs = torch.tensor(inputs)
        inputs = inputs.view(-1, NUM_FEATURES).to(DEVICE)
        targets = targets.view(-1).to(DEVICE)

        logits, probas = model.forward(inputs.view(-1, NUM_FEATURES))

        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
    
    model.eval()

    with torch.no_grad():
        
        train_loss = compute_loss(model, train_loader)
        test_loss = compute_loss(model, test_loader)
        train_acc = compute_accuracy(model, train_loader)
        test_acc = compute_accuracy(model, test_loader)

        train_acc_data.append(train_acc)
        test_acc_data.append(test_acc)
        train_cost_data.append(train_loss)
        test_cost_data.append(test_loss)

        scheduler.step(train_acc_data[-1])

        print(f"epoch: {epoch+1}/{NUM_EPOCHS}", end="")
        print(" | train_loss: %.4f" %train_loss, end = "")
        print(" | train_accuracy: %.2f" %train_acc, end = "")
        print(" || test_loss: %.4f" %test_loss, end = "")
        print(" | tset_accuracy: %.2f" %test_acc)

C:\Users\hp\AppData\Local\Temp\ipykernel_31508\64136016.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs)


epoch: 1/100 | train_loss: 1.0766 | train_accuracy: 38.89 || test_loss: 1.0783 | tset_accuracy: 38.54
epoch: 2/100 | train_loss: 1.0568 | train_accuracy: 46.30 || test_loss: 1.0565 | tset_accuracy: 50.00
epoch: 3/100 | train_loss: 1.0399 | train_accuracy: 48.84 || test_loss: 1.0458 | tset_accuracy: 53.12
epoch: 4/100 | train_loss: 1.0272 | train_accuracy: 51.39 || test_loss: 1.0362 | tset_accuracy: 56.25
epoch: 5/100 | train_loss: 1.0129 | train_accuracy: 52.55 || test_loss: 1.0297 | tset_accuracy: 56.25
epoch: 6/100 | train_loss: 0.9920 | train_accuracy: 52.78 || test_loss: 1.0125 | tset_accuracy: 53.12
epoch: 7/100 | train_loss: 0.9918 | train_accuracy: 54.86 || test_loss: 1.0113 | tset_accuracy: 54.17
epoch: 8/100 | train_loss: 0.9688 | train_accuracy: 56.02 || test_loss: 0.9878 | tset_accuracy: 55.21
epoch: 9/100 | train_loss: 0.9607 | train_accuracy: 54.86 || test_loss: 0.9798 | tset_accuracy: 55.21
epoch: 10/100 | train_loss: 0.9557 | train_accuracy: 56.25 || test_loss: 0.9788 | 

KeyboardInterrupt: 